<center>
    <h1>Randomized Search CV</h1>
</center>

Hyperparameters are parameters whose values control the learning process and determine the values of model parameters that a learning algorithm ends up learning.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from sklearn.metrics import confusion_matrix,accuracy_score

In [2]:
dataset = pd.read_csv('data.csv')
X = dataset.iloc[:, [2, 3]].values
y = dataset.iloc[:, 4].values

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset.shape

(400, 5)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state =0)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 50)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=50)

In [8]:
preds = classifier.predict(X_test)
print(accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))

0.92
[[63  5]
 [ 3 29]]


In [9]:
est = RandomForestClassifier(n_jobs=-1)
rf_p_dist={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200,300,400,500],
              'max_features':randint(1,3),
               'criterion':['gini','entropy'],
               'bootstrap':[True,False],
               'min_samples_leaf':randint(1,4),
              }

In [10]:
def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
    rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr,
                                  n_jobs=-1, n_iter=nbr_iter, cv=9)
    rdmsearch.fit(X,y)
    ht_params = rdmsearch.best_params_
    ht_score = rdmsearch.best_score_
    return ht_params, ht_score

In [11]:
rf_parameters, rf_ht_score = hypertuning_rscv(est, rf_p_dist, 40, X, y)
rf_parameters, rf_ht_score

({'bootstrap': True,
  'criterion': 'gini',
  'max_depth': None,
  'max_features': 1,
  'min_samples_leaf': 3,
  'n_estimators': 10},
 0.9121212121212121)

In [12]:
classifier=RandomForestClassifier(**rf_parameters, n_jobs=-1)

In [13]:
classifier.fit(X_train, y_train)

RandomForestClassifier(max_features=1, min_samples_leaf=3, n_estimators=10,
                       n_jobs=-1)

In [14]:
preds = classifier.predict(X_test)
print(accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))

0.94
[[64  4]
 [ 2 30]]
